# Explore USGS Terrain COGs
This 1 arc second (~30m) resolution terrain data as 32-bit cloud-optimized geotiff in a public AWS bucket.  Each file is a 1x1 degree square. 

In [ ]:
import hvplot.xarray
import fsspec
import os
import rioxarray 
from rioxarray.merge import merge_arrays

Use GDAL settings from [COG Best Practices](https://github.com/pangeo-data/cog-best-practices/blob/main/0-single-cog.ipynb)

In [ ]:
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' #This is KEY! otherwise we send a bunch of HTTP GET requests to test for common sidecar metadata
os.environ['AWS_NO_SIGN_REQUEST']='YES' #Since this is a public bucket, we don't need authentication
os.environ['GDAL_MAX_RAW_BLOCK_CACHE_SIZE']='200000000'  #200MB: Want this to be greater than size of uncompressed raster to overcome a 10 MB limit in the GeoTIFF driver for range request merging.
os.environ['GDAL_SWATH_SIZE']='200000000'  #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE
os.environ['VSI_CURL_CACHE_SIZE']='200000000' #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE

Explore a sample terrain square (the 1x1 degree square that contains most of Cape Cod)

In [ ]:
example_url = 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n42w071/USGS_1_n42w071.tif'

In [ ]:
def clip(da):
    x = da['x'].values; x0 = x.min().round(); x1 = x.max().round()
    y = da['y'].values; y0 = y.min().round(); y1 = y.max().round()
    return da.sel(x=slice(x0,x1), y=slice(y1,y0))

In [ ]:
res = 30

Try to merge images along latitude, then merge those merged images along longitude. This should allow the coordinates to align along the merged dimension

In [ ]:
prefix = 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF'
urls = []
da_lon = []
for lon in [-80, -79]:
    da_lat = []
    for lat in [41, 42]:
        url = f'{prefix}/n{lat:02d}w{-lon:03d}/USGS_1_n{lat:02d}w{-lon:03d}.tif'
        try:
            if res==30:
                da = rxr.open_rasterio(url, masked=True, chunks=True)
            else:
                level = int(res/30) - 2   # 60m = 0, 120m = 1, etc
                da = rioxarray.open_rasterio(url, masked=True, chunks=True, 
                                             overview_level=level)
            da = da.squeeze('band').sel(y=slice(None,None,-1))   # flip the y dimension so that it is ascending
            da_lat.append(clip(da))                              # clip to degree square and append
            urls.append(url)
        except:
            pass
    da_col = xr.concat(da_lat, dim='y')
    da_lon.append(da_col)

In [ ]:
prefix = 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF'
urls = []
darrays = []
for lon in [-80, -79]:
    da_lat = []
    for lat in [41, 42]:
        url = f'{prefix}/n{lat:02d}w{-lon:03d}/USGS_1_n{lat:02d}w{-lon:03d}.tif'
        try:
            if res==30:
                da = rioxarray.open_rasterio(url, masked=True, chunks=True)
            else:
                level = int(res/30) - 2   # 60m = 0, 120m = 1, etc
                da = rioxarray.open_rasterio(url, masked=True, chunks=True, 
                                             overview_level=level)
            da = da.squeeze('band', drop=True) 
            darrays.append(clip(da))                              # clip to degree square and append
            urls.append(url)
        except:
            pass

In [ ]:
da = merge_arrays(darrays)

In [ ]:
darrays[3]

In [ ]:
darrays[0].max().values

In [ ]:
da_lat

In [ ]:
da_col

In [ ]:
da_whole = xr.concat(da_lon, dim='x')

In [ ]:
da_whole.shape

Visualize with hvplot from the holoviz.org tool suite

In [ ]:
a = da_lon[0].hvplot.image(x='x', y='y', geo=True, rasterize=True, 
                frame_width=500, cmap='rainbow', tiles='ESRI')

In [ ]:
b = da_lon[1].hvplot.image(x='x', y='y', geo=True, rasterize=True, 
                frame_width=500, cmap='rainbow', tiles='ESRI')

In [ ]:
a + b

#### Explore the full list of available degree squares

In [ ]:
fs = fsspec.filesystem('s3', anon=True)

In [ ]:
flist = fs.glob('prd-tnm/StagedProducts/Elevation/1/TIFF/')

In [ ]:
flist[:5]

In [ ]:
flist[-5:]